# Think Bayes

+ [코드모음](https://github.com/AllenDowney/ThinkBayes2/tree/master/code)
+ [thinkbayes2 모듈](https://github.com/AllenDowney/ThinkBayes2/blob/master/code/thinkbayes2.py)

In [1]:
import thinkbayes2

## ch.2 계산통계 (2017.09.16)

### 2.2 쿠키문제

+ [쿠키문제링크](https://github.com/YBIGTA/Deep_learning/blob/master/ML/bayes/%5B2017.08.26.%ED%86%A0%5DThink%20Bayes_ch1.pdf)

In [2]:
from thinkbayes2 import Pmf

In [3]:
pmf = Pmf()

In [4]:
# 각 가설에 대해 사전 확률을 포함하는 분포 : 사전분포
pmf.Set('Bowl 1', 0.5)
pmf.Set('Bowl 2', 0.5)

In [5]:
#새로운 데이터(바닐라쿠키 비율)에 근거하여 이 사전분포를 갱신. 사전확률에 우도를 곱한다.
pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)

In [6]:
pmf.Normalize()

0.625

In [7]:
print('그릇1의 사후확률: %f' %pmf.Prob('Bowl 1'))
print('그릇2의 사후확률: %f' %pmf.Prob('Bowl 2'))

그릇1의 사후확률: 0.600000
그릇2의 사후확률: 0.400000


### 2.6 m&m문제

+ [m&m문제링크](https://github.com/YBIGTA/Deep_learning/blob/master/ML/bayes/%5B2017.08.26.%ED%86%A0%5DThink%20Bayes_ch1.pdf)

In [8]:
from thinkbayes2 import Suite

In [9]:
class M_and_M(Suite):
    """Map from hypothesis (A or B) to probability."""

    mix94 = dict(brown=30,
                 yellow=20,
                 red=20,
                 green=10,
                 orange=10,
                 tan=10,
                 blue=0)

    mix96 = dict(blue=24,
                 green=20,
                 orange=16,
                 yellow=14,
                 red=13,
                 brown=13,
                 tan=0)

    hypoA = dict(bag1=mix94, bag2=mix96)
    hypoB = dict(bag1=mix96, bag2=mix94)

    hypotheses = dict(A=hypoA, B=hypoB)

    def Likelihood(self, data, hypo):
        """Computes the likelihood of the data under the hypothesis.
        hypo: string hypothesis (A or B)
        data: tuple of string bag, string color
        """
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like

In [10]:
def main():
    suite = M_and_M('AB')

    suite.Update(('bag1', 'yellow'))
    suite.Update(('bag2', 'green'))

    suite.Print()


if __name__ == '__main__':
    main()

A 0.7407407407407407
B 0.2592592592592592


## ch.3 추정1 (2017.09.16)

### 3.1 주사위 문제

+ 4면체, 6면체, 8면체, 12면체, 20면체 주사위가 든 상자 
+ 상자에서 임의로 주사위 하나를 집어서 던졌더니 6이 나옴 
+ 각 주사위를 선택했을 확률은?



1. 가설 설정 
2. 데이터 설정 
3. 우도함수 작성 

In [11]:
class Dice(Suite):
    def Likelihood(self, data, hypo):
        if hypo < data:
            return 0
            #주사위를 굴려서 나온 값이 주사위의 면 수보다 크다
            #이런 경우는 없으므로 우도0
        else:
            return 1.0/hypo
            #hypo-면 주사위가 있을때 각 데이터의 값이 나올 수 있는 확률은 얼마인가?
            #확률 : 데이터에 상관없이 1/hypo

In [12]:
def main():
    suite = Dice([4, 6, 8, 12, 20])

    suite.Update(6)
    #주사위값이 6이 나왔을때 각 주사위에서 나왔을 확률
    print('After one 6')
    print('사후확률분포')
    suite.Print()
    
    print('=======================')
    
    for roll in [6, 8, 7, 7, 5, 4]:
        suite.Update(roll)

    print('After more rolls')
    print('사후확률분포')
    suite.Print()


if __name__ == '__main__':
    main()

After one 6
사후확률분포
4 0.0
6 0.3921568627450979
8 0.2941176470588235
12 0.19607843137254896
20 0.11764705882352941
After more rolls
사후확률분포
4 0.0
6 0.0
8 0.9432484536722127
12 0.055206128061290875
20 0.001545418266496554


### 3.2 기관차 문제

+ 각 철도를 지나는 기관차 1부터 N까지 순서로 번호를 붙인다.
+ 어느날 60호 기관차를 보았다.
+ 이 때 이 철도에는 몇 개의 기관차가 지나가는지 추측

1. 데이터를 보기 전에 N에 대해 알고 있는 것은? --> 사전확률  
2. N에 어떤 값이 주어졌을 때 관측한 데이터(60호 기관차)의 우도는 어떻게 되는가?

+ 가정  
> N : 1부터 1,000까지 어떤 값이든 동일한 확률로 선택

In [13]:
hypos = range(1,1001)

+ 우도함수
+ N개의 기관차 중 60호 기관차를 볼 확률은? 
+ 열차운영회사가 하나라고 가정할 때 각기관차를 볼 확률 동일. 따라서 1/N

In [14]:
class Train(Suite):
    def Likelihood(self,data,hypo):
        if hypo < data:
            return 0
            #기차가 60번까지인데 61번 기차를 볼 수는 없으니 0
        else:
            return 1.0/hypo
            #1/N

In [15]:
suite = Train(hypos)

In [16]:
#갱신함수
#데이터 : 60
suite.Update(60)

0.0028222671142652746

In [17]:
#사후확률의 평균을 계산 : 각 확률 X 전철 번호
def Mean(suite):
    total = 0
    for hypo, prob in suite.Items():
        total += hypo*prob
    return total

In [18]:
print(Mean(suite))

333.41989326371095


333대의 기관차가 지나갔다고 볼 수 있다.

In [19]:
import thinkplot

In [20]:
thinkplot.PrePlot(1)
thinkplot.Pmf(suite)
thinkplot.Save(root='train1',xlabel='Number of trains',ylabel='Probability',formats=['pdf', 'eps'])

Writing train1.pdf
Writing train1.eps


c:\program files\anaconda3\lib\site-packages\matplotlib\axes\_axes.py:545: UserWarning: No labelled objects found. Use label='...' kwarg on individual plots.
  warnings.warn("No labelled objects found. "


![train_posterior_dist](img/train1.png)

### 3.3 사전확률로 알 수 있는 것

> 사전확률을 1부터 1000까지의 균등분포로 가정  
> 사후확률의 평균은 333이었음  
> 상한값이 500 -> 사후확률 평균 207  
> 상한값이 2000 -> 사후확률평균 552

좋지 않은 추정방법 따라서 두 가지 방식으로 진행가능 (범위설정에 따라서 사후확률 평균 차이가 심함)
+ 데이터를 더 확보한다.
+ 배경지식을 더 확보한다.

#### 데이터가 더 많다면 사후확률 간 차이가 작아진다.

+ 60호 기관차에 이어서 30번과 90번 기관차도 보았다  --> 데이터 추가 확보

In [21]:
hypos500 = range(1,501)
hypos1000 = range(1,1001)
hypos2000 = range(1,2001)

In [22]:
class Train(Suite):
    def Likelihood(self,data,hypo):
        if hypo < data:
            return 0
            #기차가 60번까지인데 61번 기차를 볼 수는 없으니 0
        else:
            return 1.0/hypo
            #1/N

In [23]:
suite500 = Train(hypos500)

In [24]:
suite1000 = Train(hypos1000)

In [25]:
suite2000 = Train(hypos2000)

In [26]:
for data in [60,30,90]:
    suite500.Update(data)
    suite1000.Update(data)
    suite2000.Update(data)

In [27]:
#사후확률의 평균을 계산 : 각 확률 X 전철 번호
def Mean(suite):
    total = 0
    for hypo, prob in suite.Items():
        total += hypo*prob
    return total

In [28]:
print(Mean(suite500))
print(Mean(suite1000))
print(Mean(suite2000))

151.84958795903822
164.3055864227336
171.3381810915094


|상한선|사후평균|
|------|------|
|500   |152   |
|1,000  |164   |
|2,000  |171   |

+ 데이터 증가 : 60호 기관차만 봄 --> 30호,60호,90호 기관차 확인  
+ 상한선에 따른 사후평균의 차이가 줄어들었다.

### 3.4 사전확률의 대안

+ 데이터를 더 확보할 수 없다면 배경지식을 많이 수집하여 사전 확률을 개선하는 방식도 있음  
+ 1,000대의 기관차를 운영하는 회사라는 가정은 철도회사가 하나 뿐이라는 가정만큼 비합리적 
   
   
   
+ 철도 회사의 목록을 찾아보거나 운송 분야 전문가와 인터뷰를 통한 배경지식 확대 ... or  
+ 학습된 추측 : (ex) 대부분의 산업에서 소기업이 다수, 중견기업은 적고, 대기업은 몇 곳 없다.
  
  
    
+ 회사 규모의 분포는 로버트 액스탤(Robert Axtell)이 사이언스 지에 이고한 **멱법칙**을 따른다.  
[참고링크 : Zipf Distribution of U.S. Firm Sizes](http://science.sciencemag.org/content/293/5536/1818)

$$PMF(x) \propto (\frac{1}{x})^a  $$

+ $PMF(x)$는 x의 확률 누적함수이고 $\alpha$는 보통 1에 가깝게 설정되는 매개변수

In [29]:
#멱법칙
class Train(Dice):
    def __init__(self,hypos,alpha=1.0):
        Pmf.__init__(self)
        for hypo in hypos:
            self.Set(hypo, hypo**(-alpha))
        self.Normalize()

In [30]:
#사전확률 생성
new_hypos500 = range(1,501)
new_hypos1000 = range(1,1001)
new_hypos2000 = range(1,2001)

new_suite500 = Train(new_hypos500)
new_suite1000 = Train(new_hypos1000)
new_suite2000 = Train(new_hypos2000)

In [31]:
for data in [60,30,90]:
    new_suite500.Update(data)
    new_suite1000.Update(data)
    new_suite2000.Update(data)

In [32]:
#사후확률의 평균을 계산 : 각 확률 X 전철 번호
def Mean(suite):
    total = 0
    for hypo, prob in suite.Items():
        total += hypo*prob
    return total

In [33]:
print(Mean(new_suite500))
print(Mean(new_suite1000))
print(Mean(new_suite2000))

130.70846986256004
133.2752313750312
133.99746308073065


|상한선|사후평균|
|------|------|
|500   |131   |
|1,000  |133   |
|2,000  |134   |

+ 옅은 색 : 멱법칙 사전확률에 따른 사후확률
+ 짙은 색 : 균등분포 사전확률에 따른 사후확률
![멱법칙 사전 확률에 따른 사후확률 분포와 균등 분포 사전확률 비교](img/train4.png)

### 3.5 신뢰구간

+ 점추정 : 평균, 중간값, 최대 우도값 사용  
+ 구간추정 : 신뢰구간 정의 --> 사후분포확률을 더한 후 그 값이 5%와 95%에 해당하는 값을 기록. 즉 5분위와 95분위 값

In [34]:
def Percentile(pmf, percentage):
    p = percentage/100.0
    total = 0
    for val, prob in pmf.Items():
        total += prob
        if total >= p:
            return val

In [35]:
#멱법칙 적용, N : 1000일때 신뢰구간
interval = Percentile(new_suite1000,5), Percentile(new_suite1000,95)
print(interval)

(91, 242)


### 3.6 누적분포 함수

+ 몇 분위 이상을 계산할 때 : PMF 보다 CDF가 효율적  
+ CDF장점 : 분위수 계산이 효율적  
+ CDF : PDF와 동일한 정보, PDF <-> CDF 변경 가능  

In [36]:
cdf = new_suite1000.MakeCdf()

In [37]:
interval_from_cdf = cdf.Percentile(5), cdf.Percentile(95)

In [38]:
print(interval_from_cdf)

(91, 242)


### 3.7 독일 탱크 문제

+ 2차 대전 당시 런던의 미대사관 EWD(Economic Warfare Division) : 독일 탱크 장비 생산량 추정  
+ 탱크의 섀시와 엔진의 시리얼 번호, 기록 책, 장부, 수리 기록 확보  
+ 시리얼 번호는 제조사에 따라 할당  
+ 탱크 유형은 100개 숫자 블록  
+ 각 블록의 수가 모두 사용된 것은 아니었음  
+ 이 문제는 위의 기관차 문제와 유사  
+ 기본 정보를 파악한 미국, 영국 분석가들은 추정치 도출 : 전후 기록물에 따르면 대체로 정확한 예측

In [42]:
import pandoc